In [1]:
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import inspect
from sqlalchemy.engine import reflection

In [2]:
df = pd.read_csv('data/Stations.csv')

In [3]:
# df.head()

In [4]:
engine = create_engine("sqlite:///mta.db.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# session connection to db
session = Session(engine)

In [5]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['fareData', 'sqlite_sequence', 'stationLocationData', 'turnStileData']


### Read in Fares csv and Turnstiles Fares Sations csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [6]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfTwo = pd.read_csv('data/fares_190209.csv')

In [7]:
dfOne.head()

UNIT          STATION REMOTE                         STATION  Station ID
0  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1
1  R094     ASTORIA BLVD   R094  ASTORIA BLVD & 31ST STREET               2
2  R093            30 AV   R093  30TH AVENUE & 31ST STREET                3
3  R092         BROADWAY   R092  BROADWAY & 31ST STREET                   4
4  R091            36 AV   R091  36TH AVENUE & 31ST STREET                5

In [8]:
dfTwo.head()

REMOTE                         STATION     FF  SEN/DIS   7-D AFAS UNL  \
0   R001  WHITEHALL STREET                59035     2875            415   
1   R003  CYPRESS HILLS                    2682      147             16   
2   R004  75TH STREET & ELDERTS LANE       6865      433             61   
3   R005  85TH STREET & FOREST PKWAY       7614      562             67   
4   R006  WOODHAVEN BOULEVARD              7835      477             57   

   30-D AFAS/RMF UNL  JOINT RR TKT  7-D UNL  30-D UNL  14-D RFM UNL  \
0               1067           255    23503     37820             0   
1                 41             0     2333      1325             0   
2                138             1     4839      3680             0   
3                186             2     5007      4155             0   
4                169             0     6838      4732             0   

                                  ...                                   \
0                                 ...                                    
1                                 ...                                    
2                                 ...                                    
3                                 ...                                    
4                                 ...                                    

   PATH 2-T  AIRTRAIN FF  AIRTRAIN 30-D  AIRTRAIN 10-T  AIRTRAIN MTHLY  \
0         0         1069              0              0               0   
1         0           30              0              0               0   
2         0           51              0              0               0   
3         0           71              0              0               0   
4         0           64              0              0               0   

   STUDENTS  NICE 2-T  CUNY-120  CUNY-60  \
0      2968         0       568        0   
1       544         0        45        0   
2      2420         0        53        0   
3      1249         0       100        0   
4      1542         0       113        0   

                                                                        
0                                                ...                    
1                                                ...                    
2                                                ...                    
3                                                ...                    
4                                                ...                    

[5 rows x 29 columns]

#### Merge Both tables on remote column


In [9]:
fareData = pd.merge(dfOne, dfTwo, on='REMOTE',how="outer")

##### Set index to Station ID

In [10]:
fareData.set_index('Station ID', inplace=True)

In [11]:
# fareData.head(1)

##### Remove duplicate columns

In [12]:
fareData.drop(columns=[' STATION_x', ' STATION_y'], inplace=True)

In [13]:
# fareData.head(1)

#### Take updated fare data and add to sqllite DB

In [14]:
# fareData.to_sql('fareData', con=engine)

In [15]:
# check to see that db has both tables
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['fareData', 'sqlite_sequence', 'stationLocationData', 'turnStileData']


### Read in turnstiles_190209.csv and turnstiles_fares_stations2.csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [16]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfThree = pd.read_csv('data/turnstiles_190209.csv')

In [17]:
dfOne.head(1)

UNIT          STATION REMOTE                         STATION  Station ID
0  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1

In [18]:
dfThree.head(1)

C/A  UNIT       SCP STATION LINENAME DIVISION      DATE     TIME     DESC  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  2/2/2019  3:00:00  REGULAR   

   ENTRIES  \
0  6930882   

   EXITS                                                                 
0                                            2350417

In [19]:
turnstileData = pd.merge(dfOne, dfThree, on='UNIT')

In [20]:
turnstileData.head()

UNIT        STATION_x REMOTE                         STATION  Station ID  \
0  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
1  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
2  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
3  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
4  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   

    C/A       SCP        STATION_y LINENAME DIVISION      DATE      TIME  \
0  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019   3:00:00   
1  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019   7:00:00   
2  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  11:00:00   
3  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  15:00:00   
4  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  19:00:00   

      DESC   ENTRIES  \
0  REGULAR  10390146   
1  REGULAR  10390156   
2  REGULAR  10390285   
3  REGULAR  10390512   
4  REGULAR  10390738   

   EXITS                                                                 
0                                           12828793                     
1                                           12828909                     
2                                           12829287                     
3                                           12829923                     
4                                           12830584

In [21]:
turnstileData['ENTRIES DIFF'] = turnstileData['ENTRIES'].diff()
turnstileData.columns = turnstileData.columns.str.strip()
turnstileData['EXITS DIFF'] = turnstileData['EXITS'].diff()
turnstileData['TOTAL ACTIVITY'] = turnstileData['ENTRIES DIFF'] + turnstileData['EXITS DIFF']

In [22]:
turnstileData.head()

UNIT        STATION_x REMOTE                         STATION  Station ID  \
0  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
1  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
2  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
3  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   
4  R095  ASTORIA DITMARS   R095  DITMARS BLVD & 31ST STREET               1   

    C/A       SCP        STATION_y LINENAME DIVISION      DATE      TIME  \
0  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019   3:00:00   
1  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019   7:00:00   
2  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  11:00:00   
3  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  15:00:00   
4  R515  00-03-00  ASTORIA DITMARS      NQW      BMT  2/2/2019  19:00:00   

      DESC   ENTRIES     EXITS  ENTRIES DIFF  EXITS DIFF  TOTAL ACTIVITY  
0  REGULAR  10390146  12828793           NaN         NaN             NaN  
1  REGULAR  10390156  12828909          10.0       116.0           126.0  
2  REGULAR  10390285  12829287         129.0       378.0           507.0  
3  REGULAR  10390512  12829923         227.0       636.0           863.0  
4  REGULAR  10390738  12830584         226.0       661.0           887.0

In [23]:
turnstileData.drop(columns=['STATION_x', 'STATION_y'], inplace=True)

In [24]:
turnstileData.set_index('Station ID',inplace=True)
turnstileData.head()

UNIT REMOTE                         STATION   C/A       SCP  \
Station ID                                                                
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   

           LINENAME DIVISION      DATE      TIME     DESC   ENTRIES     EXITS  \
Station ID                                                                      
1               NQW      BMT  2/2/2019   3:00:00  REGULAR  10390146  12828793   
1               NQW      BMT  2/2/2019   7:00:00  REGULAR  10390156  12828909   
1               NQW      BMT  2/2/2019  11:00:00  REGULAR  10390285  12829287   
1               NQW      BMT  2/2/2019  15:00:00  REGULAR  10390512  12829923   
1               NQW      BMT  2/2/2019  19:00:00  REGULAR  10390738  12830584   

            ENTRIES DIFF  EXITS DIFF  TOTAL ACTIVITY  
Station ID                                            
1                    NaN         NaN             NaN  
1                   10.0       116.0           126.0  
1                  129.0       378.0           507.0  
1                  227.0       636.0           863.0  
1                  226.0       661.0           887.0

In [25]:
times = ['3:00:00', '7:00:00', '11:00:00', '15:00:00', '19:00:00',
       '23:00:00']

In [26]:
turnstileData = turnstileData.loc[turnstileData['TIME'].isin(times)]

In [27]:
turnstileData.loc[(turnstileData['ENTRIES DIFF'] < 0) | (turnstileData['ENTRIES DIFF'] > 10000), 'ENTRIES DIFF'] = 0


In [28]:
turnstileData.loc[(turnstileData['EXITS DIFF'] < 0) | (turnstileData['EXITS DIFF'] > 10000), 'EXITS DIFF'] = 0


In [29]:
turnstileData.loc[(turnstileData['TOTAL ACTIVITY'] < 0) | (turnstileData['TOTAL ACTIVITY'] > 10000), 'TOTAL ACTIVITY'] = 0


In [30]:
turnstileData.fillna(value=0,inplace=True)

In [31]:
turnstileData.head(1)

UNIT REMOTE                         STATION   C/A       SCP  \
Station ID                                                                
1           R095   R095  DITMARS BLVD & 31ST STREET      R515  00-03-00   

           LINENAME DIVISION      DATE     TIME     DESC   ENTRIES     EXITS  \
Station ID                                                                     
1               NQW      BMT  2/2/2019  3:00:00  REGULAR  10390146  12828793   

            ENTRIES DIFF  EXITS DIFF  TOTAL ACTIVITY  
Station ID                                            
1                    0.0         0.0             0.0

In [32]:
# turnstileData.to_sql('turnStileData', con=engine, chunksize=1000)

## Test queries

#### Fare data query testing


 "FF", "30-D UNL", "7-D UNL", "SEN/DIS", "7-D AFAS UNL", "30-D AFAS/RMF UNL",
    "JOINT RR TKT", "14-D RFM UNL", "1-D UNL", "14-D UNL", "7D-XBUS PASS", "TCMC",
    "RF 2 TRIP", "RR UNL NO TRADE", "TCMC ANNUAL MC", "MR EZPAY EXP", "MR EZPAY UNL", "PATH 2-T",
    "AIRTRAIN FF", "AIRTRAIN 30-D", "AIRTRAIN 10-T", "AIRTRAIN MTHLY", "STUDENTS", "NICE 2-T",
    "CUNY-120", "CUNY-60"

In [33]:
fareData = Base.classes.fareData

In [34]:
sel = [
        fareData.FF,
        fareData.SEN_DIS,
        fareData.SEVEN_D_AFAS_UNL,
        fareData.THIRTY_D_AFAS_RMF_UNL,
        fareData.JOINT_RR_TKT,
        fareData.SEVEN_D_UNL,
        fareData.THIRTY_D_UNL,
        fareData.FOURTEEN_D_RFM_UNL,
        fareData.ONE_D_UNL,
        fareData.FOURTEEN_D_UNL,
        fareData.SEVEND_XBUS_PASS,
        fareData.TCMC,
        fareData.RF_TWO_TRIP,
        fareData.RR_UNL_NO_TRADE,
        fareData.TCMC_ANNUAL_MC,
        fareData.MR_EZPAY_EXP,
        fareData.MR_EZPAY_UNL,
        fareData.PATH_TWO_T,
        fareData.AIRTRAIN_FF,
        fareData.AIRTRAIN_THIRTY_D,
        fareData.AIRTRAIN_TEN_T,
        fareData.AIRTRAIN_MTHLY,
        fareData.STUDENTS,
        fareData.NICE_TWO_T,
        fareData.CUNY_ONETWENTY,
        fareData.CUNY_SIXTY
    
    ]

In [35]:
print(session.query(*sel).filter(fareData.Station_ID == 1).all())

[(29208, 1596, 155, 263, 23, 12850, 29457, 0, 0, 0, 21, 403, 743, 16, 3585, 787, 814, 0, 251, 0, 0, 0, 2774, 0, 66, 0)]


#### Query Turnstile Data func sum entries diff, exit diff, total 

In [40]:
turnStileData = Base.classes.turnStileData

In [98]:
noFilterQuery = session.query((turnStileData.Station_ID),func.sum(turnStileData.ENTRIES_DIFF), 
                    func.sum(turnStileData.EXITS_DIFF),
                    func.sum(turnStileData.TOTAL_ACTIVITY)
                    ).group_by(turnStileData.Station_ID).all()

In [110]:
filteredQUery = session.query(      (turnStileData.DATE),
                                    (turnStileData.TIME),
                    func.sum(turnStileData.ENTRIES_DIFF), 
                    func.sum(turnStileData.EXITS_DIFF),
                    func.sum(turnStileData.TOTAL_ACTIVITY),
                              
                    ).group_by(turnStileData.DATE,turnStileData.TIME).all()

In [111]:
print(filteredQUery)

[('2/2/2019', '11:00:00', 287252.0, 223133.0, 508542.0), ('2/2/2019', '15:00:00', 430604.0, 348788.0, 776108.0), ('2/2/2019', '19:00:00', 509870.0, 411021.0, 916923.0), ('2/2/2019', '23:00:00', 282715.0, 251995.0, 532523.0), ('2/2/2019', '3:00:00', 57983.0, 147001.0, 70036.0), ('2/2/2019', '7:00:00', 68301.0, 47618.0, 115564.0), ('2/3/2019', '11:00:00', 207962.0, 157000.0, 363585.0), ('2/3/2019', '15:00:00', 352110.0, 290298.0, 639583.0), ('2/3/2019', '19:00:00', 418670.0, 341651.0, 757209.0), ('2/3/2019', '23:00:00', 204220.0, 174317.0, 376989.0), ('2/3/2019', '3:00:00', 91804.0, 99940.0, 191172.0), ('2/3/2019', '7:00:00', 47875.0, 35297.0, 82924.0), ('2/4/2019', '11:00:00', 964368.0, 806775.0, 1766614.0), ('2/4/2019', '15:00:00', 547843.0, 419819.0, 963604.0), ('2/4/2019', '19:00:00', 1140437.0, 806482.0, 1940504.0), ('2/4/2019', '23:00:00', 436769.0, 391930.0, 825996.0), ('2/4/2019', '3:00:00', 53429.0, 66963.0, 120027.0), ('2/4/2019', '7:00:00', 206118.0, 111633.0, 317003.0), ('2/5